# Ettoday 網路爬蟲實作練習


* 能夠利用 Request + BeatifulSour 撰寫爬蟲，並存放到合適的資料結構


## 作業目標

根據範例：

1. 取出今天所有的新聞
2. 取出現在時間兩小時內的新聞
3. 根據範例，取出三天前下午三點到五點的新聞

## 1. 取出今天所有的新聞

In [185]:
from selenium import webdriver
from bs4 import BeautifulSoup

browser = webdriver.Chrome(executable_path='/Users/tommy84729/python/1st-PyCrawlerMarathon/chromedriver')
browser.get("https://www.ettoday.net/news/news-list.htm")

In [192]:
import time 
from datetime import datetime, timedelta
date_time = datetime.now()
date_time = date_time.strftime('%Y/%m/%d')
date_time = datetime.strptime(date_time + ' 00', '%Y/%m/%d %H')
print("想查詢的日期：", date_time.strftime("%Y/%m/%d"))

想查詢的日期： 2020/01/23


In [198]:
time = soup.find_all(class_ = 'part_list_2')[0].find_all('h3')[-1].find(class_ = 'date').get_text()
datetime.strptime(time, '%Y/%m/%d %H:%M')
#date_time

datetime.datetime(2020, 1, 23, 13, 52)

In [202]:
import time
while True : 
    browser.execute_script("window.scrollTo(0, 100000);")
    time.sleep(0.5)
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, 'html5lib')
    time1 = soup.find_all(class_ = 'part_list_2')[0].find_all('h3')[-1].find(class_ = 'date').get_text()
    print(time1)
    if datetime.strptime(time1, '%Y/%m/%d %H:%M') < date_time : 
        print('==============')
        break
    

2020/01/23 12:00
2020/01/23 11:51
2020/01/23 11:34
2020/01/23 11:23
2020/01/23 11:16
2020/01/23 11:05
2020/01/23 10:53
2020/01/23 10:45
2020/01/23 10:30
2020/01/23 10:10
2020/01/23 10:02
2020/01/23 10:00
2020/01/23 09:49
2020/01/23 09:30
2020/01/23 09:11
2020/01/23 09:01
2020/01/23 08:22
2020/01/23 08:00
2020/01/23 07:05
2020/01/23 06:05
2020/01/23 03:01
2020/01/23 01:26
2020/01/23 01:05
2020/01/23 00:30
2020/01/23 00:10
2020/01/23 00:01
2020/01/22 23:47


In [203]:
news = []
for i in soup.find(class_="part_list_2").find_all('h3'):
    time = datetime.strptime(i.find(class_ = 'date').get_text(),'%Y/%m/%d %H:%M')
    if time > date_time : 
        news.append({
        '時間': i.find(class_ = 'date').text,
        '新聞': i.find_all('a')[0].text
        })

In [204]:
pd.DataFrame(news)

,時間,新聞
0,2020/01/23 16:33,到底怎麼做到的？極簡專家用「定額、定色」控制購物慾 包包擁有3個就夠
1,2020/01/23 16:32,快訊／俄羅斯軍方：40名敘利亞軍人慘遭突襲身亡 80人重傷
2,2020/01/23 16:30,「李柯太太」談國民黨敗選因素 「被想太多的人給耽誤」
3,2020/01/23 16:30,192cm偶像不只路雲！ 6韓團成員「超過190cm」你追了嗎？
4,2020/01/23 16:30,30歲人「骨頭像80歲」！醫揭「骨髓卡油」老更快...關節直接脆光光
...,...,...
455,2020/01/23 00:05,年節團圓郊外踏青旅行 北市交大提醒：注意長輩用路情形
456,2020/01/23 00:03,駝鹿屍體「肢解、去骨、絞碎」 阿拉斯加高中生體驗狩獵傳統文化
457,2020/01/23 00:01,《逃出絕命鎮》、《我們》班底推新作 再創種族議題「燒腦驚悚高峰」
458,2020/01/23 00:01,近10年間最狂奪冠劇本 小熊破山羊魔咒奪睽違108年冠軍


## 2. 取出現在時間兩小時內的新聞

In [111]:
datetime_two_hour = datetime.now() - timedelta(hours = 2)
datetime_two_hour

datetime.datetime(2020, 1, 23, 13, 11, 31, 320330)

In [113]:
datetime.strptime(soup.find(class_ = 'part_list_2').find_all('h3')[1].find(class_ = 'date').text,'%Y/%m/%d %H:%M' )> datetime_two_hour


True

In [115]:
news = []
for i in soup.find(class_ = 'part_list_2').find_all('h3') : 
    time = datetime.strptime(i.find(class_ = 'date').get_text(),'%Y/%m/%d %H:%M')
    if time < datetime_two_hour : 
        break
    news.append({
        '時間': i.find(class_ = 'date').text,
        '新聞': i.find_all('a')[0].text
    })
        

In [117]:
news = pd.DataFrame(news)
news

,時間,新聞
0,2020/01/23 14:30,【奧斯塔羅】刮刮樂試手氣、牌桌進財 測你年假期偏財運勢
1,2020/01/23 14:30,老公收到「酒店定位」訊息 老婆開門「見全裸小三」大戰
2,2020/01/23 14:30,張歆藝：我覺得我嫁得挺好！ 寵妻魔人袁弘3個甜蜜原則「老婆永遠是對的」
3,2020/01/23 14:30,不顧3孩子在場...台中男起邪念「強揉人妻10分鐘」！她氣喊：我不是妓女
4,2020/01/23 14:26,古力娜扎「嘴對嘴親5月大嬰兒」被罵翻！ 粉絲歪理護航：武漢肺炎傳不到新疆
5,2020/01/23 14:23,武漢「封城」回不了台灣 台商會長：3人滯留「大多選前已返鄉」
6,2020/01/23 14:23,435公斤「綠巨人浩克」公開徵婚 逾300女報名「太瘦了」全被拒
7,2020/01/23 14:20,055艦擁雙波段雷達 局座：二戰以來，中國技術首次超越美國
8,2020/01/23 14:19,中共從未放棄武力犯台 嚴德發：金門須秉持同島一命精神
9,2020/01/23 14:19,苦難球王桃田賢斗與大馬的不解之緣 挺過禁賽、車禍拼東奧金牌


## 3. 根據範例，取出三天前下午三點到五點的新聞

In [148]:
date_time = datetime.now() - timedelta(3)
print('三天前的時間：',date_time)
month = date_time.strftime('%m')
month = month.replace('0','')
day = date_time.strftime('%d')
print('月份：',month, '日期：',day)
three_days_15 = datetime.strptime(date_time.strftime("%Y/%m/%d") + ' 15', '%Y/%m/%d %H')
three_days_17 = datetime.strptime(date_time.strftime("%Y/%m/%d") + ' 17', '%Y/%m/%d %H')
print('希望查詢的時段：', three_days_15, ' - ', three_days_17)

三天前的時間： 2020-01-20 15:58:54.198730
月份： 1 日期： 20
希望查詢的時段： 2020-01-20 15:00:00  -  2020-01-20 17:00:00


In [149]:
from selenium.webdriver.support.ui import Select
selectmonth = Select(browser.find_element_by_id('selM'))
selectmonth.select_by_value(month)
selectday = Select(browser.find_element_by_id('selD'))
selectday.select_by_value(day)
browser.find_element_by_id('button').click()

In [158]:
import time
while True : 
    time.sleep(0.5)
    browser.execute_script("window.scrollTo(0, 100000);")
    html_source = browser.page_source
    soup = BeautifulSoup(html_source, 'html5lib')
    new_time = soup.find_all(class_ = 'part_list_2')[0].find_all('h3')[-1].find(class_ = 'date').get_text()
    print(new_time)
    if datetime.strptime(new_time, '%Y/%m/%d %H:%M') < three_days_15 : 
        print('==============')
        break

2020/01/20 16:44
2020/01/20 16:37
2020/01/20 16:28
2020/01/20 16:18
2020/01/20 16:10
2020/01/20 16:03
2020/01/20 15:50
2020/01/20 15:40
2020/01/20 15:30
2020/01/20 15:14
2020/01/20 15:00
2020/01/20 14:41


In [164]:
news1 = []
for i in soup.find(class_ = 'part_list_2').find_all('h3') : 
    time = datetime.strptime(i.find(class_ = 'date').get_text(),'%Y/%m/%d %H:%M')
    if three_days_15 < time < three_days_17 : 
        news1.append({
        '時間': i.find(class_ = 'date').text,
        '新聞': i.find_all('a')[0].text}) 

In [165]:
pd.DataFrame(news1)


,時間,新聞
0,2020/01/20 16:59,中職／5G智慧棒球應用 富邦悍將推出APP與球員一起春訓
1,2020/01/20 16:59,貝嫂愛用英國超夯潤唇筆 連乾裂死皮都能立即神救援
2,2020/01/20 16:57,排隊買奶茶！5歲童手放櫥窗遭電擊...5根指頭全燒黑 媽嚇壞：聞到焦味
3,2020/01/20 16:56,成功外海漁船失火竄濃煙 2船員跳海被救上膠筏無傷亡
4,2020/01/20 16:55,嘉義縣市愛心年菜暨服務經費募捐 尚有一成缺口
...,...,...
109,2020/01/20 15:08,全非洲最富有的女人伊莎貝爾 靠著總統老爸擁有660億資產
110,2020/01/20 15:04,飆風玫瑰連7場砍20+5平生涯紀錄 活塞主帥：羅斯該入選全明星賽
111,2020/01/20 15:03,「在不瘋狂」前團員爆料...游否希爆氣「不要當我塑膠」：有人想暗殺團員！
112,2020/01/20 15:03,馬尾妹旅館洗澡滑倒…下秒「雙腿突鑽出」血色嬰屍！她全臉問號：我沒懷孕啊


In [205]:
browser.close()